In [1]:
%run "C:\Repos\AICourse\PersonalFinanceAgent\agents\email_classifier_agent.ipynb"

Using Azure OpenAI service
Kernel created successfully for the email claissifier agent!
Initializing EmailPlugin.
Service settings updated: Temperature set to 0.7
Agent 'EmailClassifierAgent' created successfully!


In [2]:
from semantic_kernel.agents import ChatHistoryAgentThread

In [3]:
import ast
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent

# Create a chat function
async def chat(agent, thread, question):
    print(f"\nUser: {question}")

    # Use invoke to see the full process including function calls
    function_calls = []
    function_results = []

    response = await agent.get_response(messages=question, thread=thread)
    thread = response.thread

    # Fix: Use async for to iterate through the async generator
    final_response = None
    async for r in thread.get_messages():
        for item in r.items:
            if isinstance(item, FunctionCallContent):
                function_call = item
                function_calls.append(function_call)
                print(f"\n{80 * '-'}")
                print(f"Function call: {function_call.function_name}")
                print(f"Arguments: {function_call.arguments}")
            elif isinstance(item, FunctionResultContent):
                function_result = item
                function_results.append(function_result)
                print(f"Function result: {function_result.result}")
                print(80 * "-")

        final_response = r

    # Get the final answer separately for clarity
    start = final_response.content.find('[')
    end = final_response.content.rfind(']') + 1
    email_dict_str = final_response.content[start:end]

    # Parse the string into a Python dictionary
    classified_emails = ast.literal_eval(email_dict_str)
    print(f"Agent: \n{classified_emails}\n")

    return {
        "function_calls": function_calls,
        "function_results": function_results,
        "final_response": final_response,
    }

In [4]:
# Test with different questions
questions = [
    "Please fetch and classify all my emails.",
]

# Test each question
for question in questions:
    # Create a chat history
    thread: ChatHistoryAgentThread = None
    await chat(email_classifier_agent, thread, question)


User: Please fetch and classify all my emails.
Reading demo data file.
Demo data file read successfully.

--------------------------------------------------------------------------------
Function call: get_user_emails
Arguments: {}
Function result: [{'subject': '[IMP]: Your Credit Score has ARRIVED!', 'sender': 'Team BankBazaar <creditreport+ratealert@bankbazaar.com>', 'date': '01 Oct 2024 12:27:04 +0530 (IST)', 'body': '', 'id': 1}, {'subject': 'Transaction on XYZ bank account', 'sender': 'tx@xyz.com', 'date': '04 Oct 2024 18:02:54 +0530', 'body': 'Transaction on XYZ bank account Rs. 900.00 on Oct 04, 2024 at 18:02:54. Info: ABC SUPERMARKET.', 'id': 2}, {'subject': "Feb mutual fund investment data, L&T's large contract, & more - Groww Digest", 'sender': 'Groww Digest <noreply@digest.groww.in>', 'date': '11 Oct 2024 02:15:45 +0530', 'body': '', 'id': 3}, {'subject': 'Transaction on XYZ bank account', 'sender': 'tx@xyz.com', 'date': '06 Oct 2024 13:28:49 +0530', 'body': 'Transaction on